# Setup

Make sure to choose GPU under Edit -> Notebook settings -> Hardware accelerator

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [3]:
data = pd.read_csv("train_cleaned.csv")

data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


In [4]:
print((data.toxic == 1).sum()) # toxic
print((data.toxic == 0).sum()) # non toxic

13934
135414


# Tokenizing Comments using TextVectorization

In [5]:
# find the maximum number of words present in any given comment

maxlen = 0
longest_comment = ""
for comment in data['comment_text']:
    length = len(comment)
    if (length > maxlen):
        longest_comment = comment
    maxlen = max(maxlen, length)

print("Number of characters in the longest comment is", maxlen)
print("Number of words in the longest comment is",
      len(longest_comment.split(" ")) + 1)

Number of characters in the longest comment is 5000
Number of words in the longest comment is 456


In [6]:
vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(

    max_tokens=None,

    # this is greater than the max words any comment has (774)
    # the remaning spots in the output would be padded by 0s
    output_sequence_length=800,

    # converets to lowercase and skips all the punctuation
    standardize="lower_and_strip_punctuation",

    # the tokens will be split at whitespaces
    split="whitespace",

    # each of the tokens is represented as an integer
    output_mode="int",
)

In [7]:
numpyArray = data[data.columns[1]].to_numpy()
vectorize_layer.adapt(numpyArray)

In [8]:
# testing
vectorize_layer("hello, world!")

<tf.Tensor: shape=(800,), dtype=int64, numpy=
array([185, 161,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,  

# Creating a Tokenizer

### Gets all the words into one string

In [9]:
# Creates the tokenizer class
tokenizer = keras.preprocessing.text.Tokenizer()

# Combines all the words into one singular string
allWordsString = " ".join(data.head(50)["comment_text"].tolist())
allWordsList = allWordsString.split(r"\\s+")

# Updates the tokenizer with the string of all words
tokenizer.fit_on_texts(allWordsList)

# Prints word dictionary
# print(tokenizer.word_index)

# Prints length of word dictionary
print(len(tokenizer.word_index))

# Converts text to numbers
print(tokenizer.texts_to_sequences(["page", "im", "use", "mussolini"]))

1077
[[1], [2], [3], [1077]]


# Text Vectorization Layer

Reference [here](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

In [10]:
NUM_ROWS = 10000

MAX_LENGTH = None
encoder = tf.keras.layers.TextVectorization(
    output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'article', 'page', 'wikipedia', 'would', 'talk',
       'like', 'one', 'please', 'dont', 'see', 'also', 'im', 'know',
       'think', 'edit', 'people', 'use', 'articles'], dtype='<U322')

## Testing the Encoder

Encoder removes punctuation and whitespace and forces lowercase so half the cleaning we did was useless

In [11]:
commentsToEncode = data.head(3)["comment_text"]
print(commentsToEncode)

encodedComments = encoder(commentsToEncode).numpy()
print(encodedComments)

for comment in encodedComments:
    print(" ".join(vocab[comment]))

0    explanation\n  edits made   username hardcore ...
1    daww  matches  background colour im seemingly ...
2    hey man im really  trying  edit war     guy  c...
Name: comment_text, dtype: object
[[  530    53    41   437  4205 14587   257   208  2022  9650  6733  4903
   2845    44  1016 11400  2690     9    10   157   357     6     3    56
     13  3853]
 [35388  2050  1647  6725    13  3256  2365    26     6   977    95     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0]
 [  387   349    13    49   169    16   218   524  1606   406   367    30
    399    53   270     6     3   126   341  2062   506   391     0     0
      0     0]]
explanation edits made username hardcore metallica fan reverted werent vandalisms closure gas voted new york dolls fac please dont remove template talk page since im retired
daww matches background colour im seemingly stuck thanks talk january utc               
hey man im really trying edit war guy constantly

## Building the Model

In [12]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)

regularization_layer = tf.keras.layers.Dense(
    64, 
    activation="relu", 
    #kernel runs before activation, activity runs after
    kernel_regularizer=tf.keras.regularizers.l1(0.001),
    activity_regularizer=tf.keras.regularizers.l2(0.001)
)

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])

## Training the Model

In [13]:
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

## Early Stopping

to prevent overfitting during the training process

In [14]:
# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=2)

history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback])

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


249/250 [============================>.] - ETA: 0s - loss: 0.4315 - accuracy: 0.8843

/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


250/250 [==============================] - 79s 209ms/step - loss: 0.4309 - accuracy: 0.8842 - val_loss: 0.2524 - val_accuracy: 0.9175
Epoch 2/10
250/250 [==============================] - 37s 149ms/step - loss: 0.1995 - accuracy: 0.9360 - val_loss: 0.1742 - val_accuracy: 0.9365
Epoch 3/10
250/250 [==============================] - 29s 115ms/step - loss: 0.0800 - accuracy: 0.9776 - val_loss: 0.1880 - val_accuracy: 0.9365
Epoch 4/10
250/250 [==============================] - 25s 99ms/step - loss: 0.0251 - accuracy: 0.9936 - val_loss: 0.2284 - val_accuracy: 0.9415
Epoch 5/10
250/250 [==============================] - 21s 85ms/step - loss: 0.0089 - accuracy: 0.9990 - val_loss: 0.2548 - val_accuracy: 0.9460
Epoch 6/10
250/250 [==============================] - 20s 81ms/step - loss: 0.0041 - accuracy: 0.9996 - val_loss: 0.2862 - val_accuracy: 0.9450
Epoch 7/10
250/250 [==============================] - 18s 71ms/step - loss: 0.0029 - accuracy: 0.9999 - val_loss: 0.3151 - val_accuracy: 0.9460


## Getting Test Data

In [15]:
test_data = pd.read_csv("test_cleaned.csv")
test_labels = pd.read_csv("test_labels.csv")

test_labels = test_labels.loc[test_labels["toxic"] >= 0]
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [16]:
# Tests all rows with a value of 0 or 1

test_df = merged_df[["comment_text", "toxic"]]
testTarget = test_df.pop("toxic")
model.evaluate(test_df, testTarget)

1824/1824 [==============================] - 33s 18ms/step - loss: 0.5244 - accuracy: 0.9054


[0.5243768095970154, 0.9053542613983154]

In [17]:
# Tests only rows with a toxic value of 1

test_df = merged_df[["comment_text", "toxic"]]
newTest_df = test_df.loc[test_df["toxic"] == 1]

newTestTarget = newTest_df.pop("toxic")
model.evaluate(newTest_df, newTestTarget)

163/163 [==============================] - 3s 20ms/step - loss: 2.7107 - accuracy: 0.6152


[2.710728883743286, 0.6152366399765015]